# **Codig Dojo**: reto kaggle

- Fecha de creación: 2022-01-22
- Fecha de modificación: 2022-01-16

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from linearmodels import OLS

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


import datawig
plt.rcParams['figure.dpi'] = 100


In [67]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df2 = pd.concat([df_train, df_test], keys=[0,1])

In [69]:
print(df_train.shape, df_test.shape)

(1460, 81) (1459, 80)


In [68]:
df2

Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0 0        1          60       RL         65.0     8450   Pave   NaN      Reg   
  1        2          20       RL         80.0     9600   Pave   NaN      Reg   
  2        3          60       RL         68.0    11250   Pave   NaN      IR1   
  3        4          70       RL         60.0     9550   Pave   NaN      IR1   
  4        5          60       RL         84.0    14260   Pave   NaN      IR1   
...      ...         ...      ...          ...      ...    ...   ...      ...   
1 1454  2915         160       RM         21.0     1936   Pave   NaN      Reg   
  1455  2916         160       RM         21.0     1894   Pave   NaN      Reg   
  1456  2917          20       RL        160.0    20000   Pave   NaN      Reg   
  1457  2918          85       RL         62.0    10441   Pave   NaN      Reg   
  1458  2919          60       RL         74.0     9627   Pave   NaN      Reg   

       LandContour Utilities  ... PoolArea PoolQC  Fence MiscFeature MiscVal  \
0 0            Lvl    AllPub  ...        0    NaN    NaN         NaN       0   
  1            Lvl    AllPub  ...        0    NaN    NaN         NaN       0   
  2            Lvl    AllPub  ...        0    NaN    NaN         NaN       0   
  3            Lvl    AllPub  ...        0    NaN    NaN         NaN       0   
  4            Lvl    AllPub  ...        0    NaN    NaN         NaN       0   
...            ...       ...  ...      ...    ...    ...         ...     ...   
1 1454         Lvl    AllPub  ...        0    NaN    NaN         NaN       0   
  1455         Lvl    AllPub  ...        0    NaN    NaN         NaN       0   
  1456         Lvl    AllPub  ...        0    NaN    NaN         NaN       0   
  1457         Lvl    AllPub  ...        0    NaN  MnPrv        Shed     700   
  1458         Lvl    AllPub  ...        0    NaN    NaN         NaN       0   

       MoSold YrSold  SaleType  SaleCondition  SalePrice  
0 0         2   2008        WD         Normal   208500.0  
  1         5   2007        WD         Normal   181500.0  
  2         9   2008        WD         Normal   223500.0  
  3         2   2006        WD        Abnorml   140000.0  
  4        12   2008        WD         Normal   250000.0  
...       ...    ...       ...            ...        ...  
1 1454      6   2006        WD         Normal        NaN  
  1455      4   2006        WD        Abnorml        NaN  
  1456      9   2006        WD        Abnorml        NaN  
  1457      7   2006        WD         Normal        NaN  
  1458     11   2006        WD         Normal        NaN  

[2919 rows x 81 columns]

In [3]:
perdidos = df_train.isna().sum()/df_train.shape[0]

In [57]:
perdidos.to_clipboard()
perdidos

Id               0.000000
MSSubClass       0.000000
MSZoning         0.000000
LotFrontage      0.177397
LotArea          0.000000
                   ...   
MoSold           0.000000
YrSold           0.000000
SaleType         0.000000
SaleCondition    0.000000
SalePrice        0.000000
Length: 81, dtype: float64

In [5]:
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [6]:
df_train.shape

(1460, 81)

In [93]:
def limpiar(df):
    dft = df.drop(['Alley', 'FireplaceQu', 'PoolQC', 'Fence',
            'MiscFeature', 'LotFrontage'], axis=1)
        
    sub = [i for i in dft.columns if 'SalePrice' not in i ]
    return dft #.dropna(subset=sub)


# intento 1

In [8]:
df = limpiar(df_train)

In [9]:
df.dtypes.to_clipboard()

In [10]:
df.corr()['SalePrice'].to_clipboard()

In [11]:
yvar = 'SalePrice'
xvar = list(df.select_dtypes(['int','float']).columns)[1:-1]

eliminadas = 'TotalBsmtSF'
xvar.remove(eliminadas)

In [12]:
xvar = ['OverallQual','GrLivArea','GarageCars','GarageArea','1stFlrSF','TotalBsmtSF','FullBath','TotRmsAbvGrd','YearBuilt','YearRemodAdd',]

In [13]:
formula = yvar+'~1+'+'+'.join(xvar)

In [14]:
ols1 = OLS.from_formula(formula=formula, data=df).fit()

In [15]:
ols1

OLS Estimation Summary                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                      0.7658
Estimator:                        OLS   Adj. R-squared:                 0.7641
No. Observations:                1338   F-statistic:                    2143.5
Date:                Mon, Feb 21 2022   P-value (F-stat)                0.0000
Time:                        19:52:31   Distribution:                 chi2(10)
Cov. Estimator:                robust                                         
                                                                              
                              Parameter Estimates                               
================================================================================
              Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------
Intercept    -1.191e+06  1.398e+05    -8.5194     0.0000  -1.465e+06  -9.169e+05
1stFlrSF         11.123     16.640     0.6684     0.5039     -21.491      43.737
FullBath        -9688.3     4345.6    -2.2295     0.0258  -1.821e+04     -1171.1
GarageArea       16.210     19.670     0.8241     0.4099     -22.342      54.762
GarageCars    1.267e+04     7927.9     1.5980     0.1100     -2869.4   2.821e+04
GrLivArea        51.092     16.234     3.1473     0.0016      19.275      82.910
OverallQual    2.09e+04     2009.2     10.403     0.0000   1.696e+04   2.484e+04
TotRmsAbvGrd     1070.3     2513.6     0.4258     0.6703     -3856.3      5996.9
TotalBsmtSF      22.343     24.810     0.9006     0.3678     -26.283      70.969
YearBuilt        246.15     46.797     5.2599     0.0000      154.43      337.87
YearRemodAdd     313.36     70.367     4.4533     0.0000      175.45      451.28
================================================================================
OLSResults, id: 0x298d66db5b0

In [16]:
pred1 = ols1.predict(data=df_test.fillna(0))
pred1['Id'] = df_test.Id
pred1 = pred1.set_index('Id').rename(columns={})
pred1.to_csv('pred1.csv')

# intento 2

In [121]:
df3 = pd.get_dummies(limpiar(df2.drop('Id', axis=1))).reset_index().drop('level_1', axis=1)

df_rf = df3[df3['level_0']==0].drop('level_0', axis=1).dropna()
df_rf_test = df3[df3['level_0']==1].drop('level_0', axis=1).drop('SalePrice', axis=1)

In [115]:
X = df_rf.drop('SalePrice', axis=1) #.values.reshape(-1, 1)
y = df_rf['SalePrice']

In [116]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1357)

In [117]:
est_range = [10,50,100]
scores = []
for est in est_range:
    rf = RandomForestRegressor(random_state=13579, bootstrap=True, n_estimators=est)
    rf.fit(X_train, y_train)
    scores.append(rf.score(X_test, y_test))
    

In [118]:
scores

[0.8720572119159022, 0.8827674965501731, 0.8809669376245909]

In [119]:
rf = RandomForestRegressor(random_state=13579, bootstrap=True, n_estimators=50)
rf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=50, random_state=13579)

In [130]:
pred2 = rf.predict(df_rf_test.fillna(0))
pred2 = pd.DataFrame(pred2)
pred2['Id'] = df_test.Id
pred2.to_csv('pred2.csv')